# My hangman file

In [1]:
import json
import numpy as np
import math
import requests
import sklearn
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams, FreqDist
import sklearn.model_selection
import matplotlib.pyplot as plt

Ok, I think I am almost done, I just want to add in functionality where I can give the class `test_filename` and `train_filename` and it correctly loads those as test and train datasets. Right now it can only load 1 dataset.

In [31]:

text_file = open("words_250000_train.txt","r")
full_dict = text_file.read().splitlines()
text_file.close()


In [32]:
full_dict

['aaa',
 'aaaaaa',
 'aaas',
 'aachen',
 'aaee',
 'aag',
 'aahed',
 'aahs',
 'aal',
 'aalesund',
 'aaliis',
 'aalst',
 'aam',
 'aandahl',
 'aao',
 'aapss',
 'aar',
 'aarau',
 'aardvark',
 'aardwolf',
 'aaren',
 'aargh',
 'aarika',
 'aaronic',
 'aaronite',
 'aaronsburg',
 'aarp',
 'aarrghh',
 'aas',
 'aasvogels',
 'aaup',
 'aavso',
 'aba',
 'ababdeh',
 'abac',
 'abacas',
 'abacaxi',
 'abaci',
 'abacination',
 'abaciscus',
 'aback',
 'abaco',
 'abacterial',
 'abactinally',
 'abactor',
 'abaculus',
 'abacuses',
 'abada',
 'abaddon',
 'abadengo',
 'abadite',
 'abaft',
 'abagail',
 'abailard',
 'abaised',
 'abaisse',
 'abaka',
 'abakas',
 'abalation',
 'abalienated',
 'abalienation',
 'abalones',
 'abamp',
 'abamperes',
 'abana',
 'abandon',
 'abandoned',
 'abandonee',
 'abandoners',
 'abandonment',
 'abandons',
 'abanet',
 'abanic',
 'abantes',
 'abaptiston',
 'abarambo',
 'abaris',
 'abarticular',
 'abas',
 'abased',
 'abasedness',
 'abasements',
 'abasers',
 'abasgi',
 'abashed',
 'abashe

In [17]:
class MyHangman(object):
    def __init__(self, weight_length = 1, weight_count = 0.5, correct_word = "", training_dictionary_location = "words_250000_train.txt", test_dictionary_location = "", train = True, test_size = 0.33):
        self.weight_length = weight_length
        self.weight_count = weight_count
        self.train = train

        self.guessed_letters = []
        if correct_word != "" :
            self.givenExternalWord = True
        else:
            self.givenExternalWord = False
        
        self.word = ""
        self.correct_word = correct_word

        self.training_dictionary, self.test_dictionary = self.build_dictionaries(training_dictionary_location, test_dictionary_location, test_size)

        self.allNGrams = self.generateNGrams(self.training_dictionary, 5)
        self.current_dictionary = []
        self.alphabet = list(string.ascii_lowercase)
        self.vowels = list(["a", "e", "i", "o", "u"])
        self.guess_list = []
        self.has_won = False
        self.gameLog = []
        #self.init_grams()
        
    def build_dictionaries(self, train_loc, test_loc, test_size):
        if(self.train == True):
            #If we are in training mode, let's assume that we have one dictionary we are dividing in a test/train split.
            #if we are in training mode, it would make sense that this is training_dictionary_location...
            full_dict = self.build_dictionary(train_loc)
            dict_train, dict_test = sklearn.model_selection.train_test_split(full_dict, test_size = test_size) #test_size is adjustable, it doesn't do anything if not training though
        else:
            #else load both dicts directly
            dict_train = self.build_dictionary(train_loc)   
            dict_test = self.build_dictionary(test_loc)

        return [dict_train, dict_test]

    def change_weights(self, wl, wc):
        self.weight_length = wl
        #self.weight_probabolity = wp
        self.weight_count = wc
        #can probably mess with more weights...


    def generateNGrams(self, vocab, n):
        #return the ngrams as dictionaries
        #I used the freq_dists and ngrams from nltk but they are way too slow
        #this is a little confusing because I am loading words from a dictionary (not the data type) of words...
        #let's call it vocab here instead
        allNGrams = []
        for k in range(2, n+1):
            ngrams = {}
            for word in vocab:
                word = "." + word + "." # pad with start and end bits
                if len(word) < k:
                    continue
                for i in range(len(word) - k + 1):
                    gram = word[i:i+k]
                    if gram not in ngrams:
                        ngrams[gram] = 1
                    else:
                        ngrams[gram] += 1
            allNGrams.append(ngrams.copy())
        return allNGrams
        
        
    def generate_correct_word(self):
        #Ok, so by default the code will generate a random correct word from the full dictionary
        #You can also initialize the class with an external correct word, or set one after initialization
        if self.givenExternalWord == False:
            self.correct_word = random.choice(self.test_dictionary)
    
    def set_correct_word(self, givenWord):
        self.correct_word = givenWord
        self.givenExternalWord = True

    def reset_correct_word(self):
        #this is just to reset the given correct word in case that is necessary
        self.correct_word = ""
        self.givenExternalWord = False


    def find_potential_guesses(self, allGrams, input_word, guess_list):
        
        potential_guesses = []
        remove_set = set(guess_list)
        remove_set.add(None)

        substr_list = []
        input_word = "." + input_word + "." #pad beginning and end with something to indicate beginning/end

        for i,char in enumerate(input_word):
            if char == "_": #found a blank
                start, end = i, i
                #print(i)
                #get longest substring containing the blank
                while start > 0 and input_word[start-1] != "_":
                    start-=1
                    #print(start)
                while end < len(input_word) - 1 and input_word[end+1] != "_":
                    end += 1
                    #print(end)
                substring = input_word[start : end + 1]
                substr_list.append(substring)
        #print(substr_list)

        for substr in substr_list:
            max_dim = min(len(substr), 5)
            current_dim = max_dim
            match_temp_array = []

            while current_dim >= 2:
                ngramDict = allGrams[current_dim - 2]

                grams = ngrams(substr, current_dim)
                subsubstr_list = ["".join(g) for g in grams if g.count("_") == 1]
                subsubstr_list = [x for x in subsubstr_list if (x != "_." and x != "._")]
                #print(subsubstr_list)
                
                for subsubstr in subsubstr_list:
                    #print(subsubstr)
                    #print(ngramDict)
                    idx = subsubstr.find("_")
                    total_match_count = 0
                    for l in string.ascii_lowercase:
                        filledSubstr = subsubstr[0:idx] + l + subsubstr[idx+1::]
                        #print(filledSubstr)
                        if(ngramDict.get(filledSubstr)):
                            count = ngramDict[filledSubstr]
                            match_temp_array.append([l, filledSubstr, count])
                            total_match_count += count
                            #print(l, count, filledSubstr, sep = " ")
                    for item in match_temp_array:
                        if (item[0] not in guess_list and total_match_count > 0):
                            potential_guesses.append([item[0], current_dim, item[2] / total_match_count, total_match_count])
                
                #print(match_temp_array)
                if(len(match_temp_array) > 0):
                    #print(match_temp_array)
                    #print(potential_guesses)
                    break

                current_dim -= 1
        return potential_guesses
        

    def evaluate_potential_guesses(self, potential_guesses):
        #print(potential_guesses)
        if(potential_guesses == []):
            return "!" # no n-gram fits
        # Initialize dictionary to keep scores for each guess
        scores = {}

        # Weighting factors, these are the main thing to tweak...
        weight_length = self.weight_length  # Giving more weight to the length of the n-gram
        weight_count = self.weight_count

        for guess, length, probability, tot_count in potential_guesses:
            guess_letter = list(guess)[0]  # Extract the letter from the set
            #if length == 2:
            #    continue

            # Calculate the score for this guess
            #score = probability * weight_probability + (length * weight_length) + weight_count * np.log(tot_count)
            score = np.log(probability * tot_count) * weight_count + weight_length * length

            # Add or update the score for this letter in the scores dictionary
            if guess_letter in scores:
                scores[guess_letter] += score
            else:
                scores[guess_letter] = score

        # Determine the best guess by finding the maximum score
        #print(scores)
        if len(scores) == 0:
            return "!"
        best_guess = max(scores, key=scores.get)
        return best_guess



    def guess(self, word): # word input example: "_ p p _ e "

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        #clean_word = word.replace("_",".") #I am using underscores

        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        #print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        reMatchWord = clean_word.replace("_",".")
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(reMatchWord, dict_word):
                new_dictionary.append(dict_word)


        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()
        #print(sorted_letter_count)                
        
        #guess_letter = '!'

        all_guesses = self.find_potential_guesses(self.allNGrams, word, self.guessed_letters)
        #print(all_guesses)
        ngram_guess = self.evaluate_potential_guesses(all_guesses)
        
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        #print(sorted_letter_count)
        most_common_guess = "!"

        for letter, instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                most_common_guess = letter
                #print(most_common_guess)
                break
        
        if(ngram_guess != "!"):
            return ngram_guess
        if(most_common_guess != "!"):
            return most_common_guess
        
        #ok, so we will only get down here if the current dictionary is EMPTY and there are no ngram matches...
        #what do we do in this case... well let's filter based on the letters
        #It is rare that this happens...
        backup_dictionary = self.training_dictionary

        last_dictionary = backup_dictionary.copy()

        incorrect_set = set(self.guessed_letters) - set(word)
        #correct_set = set(self.guessed_letters) - incorrect_set

        backup_dictionary = [word for word in backup_dictionary if set(word).isdisjoint(incorrect_set)]
        backup_dict_string = "".join(backup_dictionary)
        bc = collections.Counter(backup_dict_string)
        backup_letter_count = bc.most_common()

        backup_guess = "!"
        for letter, instance_count in backup_letter_count:
            if letter not in self.guessed_letters:
                backup_guess = letter
                #print(most_common_guess)
                break
        if backup_guess != "!":
            return backup_guess
        
        #Ok... but what if this also fails?
        #well... let's just default back to the order of the entire dictionary
        #I guess in some freak accident the code could be given a training dictionary where there were only 5 letters in all of the words and the test word has one that isn't there... but that would be pretty insane.
        last_dict_string = "".join(last_dictionary)
        lc = collections.Counter(last_dict_string)
        last_letter_count = lc.most_common()

        for letter, instance_count in last_letter_count:
            if letter not in self.guessed_letters:
                last_guess = letter
                #print(most_common_guess)
                break
        
        return last_guess


    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word

    def get_winstate(self):
        return self.has_won
    
    def get_gamelog(self):
        return self.gameLog
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.gameLog = []
        self.guessed_letters = []
        self.current_dictionary = self.training_dictionary
        self.has_won = False
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        
        self.gameLog.append([self.correct_word, word, self.guessed_letters.copy(), tries_remains])
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            

            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                self.gameLog.append([self.correct_word, word, self.guessed_letters.copy(), tries_remains])
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))

            self.gameLog.append([self.correct_word, word, self.guessed_letters.copy(), tries_remains])
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                



So... implementing a "probability" actually decreased how good the model is which is kinda sad...

Also, even after messing with the code, it should be the same result as in 6 but it's not...

So gotta fix that...

maybe the issue is the 2-grams... idk maybe it's better to start at 3...

In [9]:
game = MyHangman(weight_count = 0.25, weight_length = 2)
game.set_correct_word("ptilo")

In [36]:
game.start_game()
print()

Successfully start a new game! # of tries remaining: 6. Word: _____.
Guessing letter: a
Incorrect guess a, # of tries remaining: 6. Word: _____.
Guessing letter: e
Incorrect guess e, # of tries remaining: 5. Word: _____.
Guessing letter: s
Incorrect guess s, # of tries remaining: 4. Word: _____.
Guessing letter: o
Got a Letter, o, # of tries remaining: 3. Word: ____o.
Guessing letter: t
Got a Letter, t, # of tries remaining: 3. Word: _t__o.
Guessing letter: i
Got a Letter, i, # of tries remaining: 3. Word: _ti_o.
Guessing letter: n
Incorrect guess n, # of tries remaining: 3. Word: _ti_o.
Guessing letter: c
Incorrect guess c, # of tries remaining: 2. Word: _ti_o.
Guessing letter: p
Got a Letter, p, # of tries remaining: 1. Word: pti_o.
Guessing letter: l
Success! the word was: ptilo



In [14]:
game = MyHangman(weight_count = 0.25, weight_length = 2, train = True)
wincount = 0
N = 1000
for i in range(N):
    if i % (N/10) == 0:
        print("on game %i of %i" % (i+1, N))
        win_percent = 100*wincount/(i+1)
        print("win percentage: %f" % win_percent)
    game.start_game(verbose=False)
    wincount += game.get_winstate()
print()
print("win percentage: ", 100*wincount/N, "%")

on game 1 of 1000
win percentage: 0.000000
on game 101 of 1000
win percentage: 55.445545
on game 201 of 1000
win percentage: 52.238806
on game 301 of 1000
win percentage: 52.159468
on game 401 of 1000
win percentage: 49.376559
on game 501 of 1000
win percentage: 48.902196
on game 601 of 1000
win percentage: 49.251248
on game 701 of 1000
win percentage: 49.500713
on game 801 of 1000
win percentage: 49.937578
on game 901 of 1000
win percentage: 50.832408

win percentage:  50.8 %


In [18]:
game = MyHangman(train = True)
gridScores = []
weight_count_vector = np.linspace(0.1, 0.5, 11)
weight_length_vector = np.linspace(1, 5, 11)
M = len(weight_count_vector) * len(weight_length_vector)
#MyHangman(weight_count = 0.25, weight_length = 2) best performance so far...

tr = 0
for wc in weight_count_vector:
    for wl in weight_length_vector:
        tr += 1
        game.change_weights(wl, wc)
        wincount = 0
        N = 1000
        for i in range(N):
            #if i % (N/10) == 0:
                #print("on game %i of %i" % (i+1, N))
                #win_percent = 100*wincount/(i+1)
                #print("win percentage: %f" % win_percent)
            game.start_game(verbose=False)
            wincount += game.get_winstate()
        print("Finished trial %i of %i" % (tr, M))
        print("win percentage: ", 100*wincount/N, "%")
        print()
        gridScores.append([wl, wc, 100*wincount/N])

Finished trial 1 of 121
win percentage:  49.6 %

Finished trial 2 of 121
win percentage:  51.6 %

Finished trial 3 of 121
win percentage:  53.5 %

Finished trial 4 of 121
win percentage:  53.3 %

Finished trial 5 of 121
win percentage:  51.1 %

Finished trial 6 of 121
win percentage:  50.7 %

Finished trial 7 of 121
win percentage:  53.4 %

Finished trial 8 of 121
win percentage:  50.4 %

Finished trial 9 of 121
win percentage:  55.6 %

Finished trial 10 of 121
win percentage:  53.6 %

Finished trial 11 of 121
win percentage:  51.3 %

Finished trial 12 of 121
win percentage:  54.4 %

Finished trial 13 of 121
win percentage:  49.5 %

Finished trial 14 of 121
win percentage:  50.2 %

Finished trial 15 of 121
win percentage:  52.9 %

Finished trial 16 of 121
win percentage:  52.3 %

Finished trial 17 of 121
win percentage:  53.5 %

Finished trial 18 of 121
win percentage:  52.2 %

Finished trial 19 of 121
win percentage:  49.5 %

Finished trial 20 of 121
win percentage:  49.1 %

Finished 

In [19]:
gridScores.sort(key = lambda x: x[2], reverse = True)
top_20 = gridScores[0:20]
print(top_20)

[[4.2, 0.45999999999999996, 56.9], [5.0, 0.26, 55.7], [1.4, 0.42000000000000004, 55.7], [4.2, 0.1, 55.6], [1.0, 0.42000000000000004, 55.3], [4.6, 0.5, 55.3], [1.4, 0.45999999999999996, 55.2], [4.2, 0.22, 54.8], [2.6, 0.26, 54.8], [1.8, 0.30000000000000004, 54.6], [3.4000000000000004, 0.30000000000000004, 54.5], [1.0, 0.14, 54.4], [3.0, 0.45999999999999996, 54.4], [3.0, 0.18, 54.2], [5.0, 0.18, 54.2], [1.4, 0.22, 54.1], [3.0, 0.30000000000000004, 54.1], [4.2, 0.30000000000000004, 54.1], [2.6, 0.45999999999999996, 54.1], [4.2, 0.26, 54.0]]


In [42]:
#top_20 = [[3.0, 0.45, 64.9], [0.5, 0.1, 64.1], [2.5, 0.45, 64.1], [3.5, 0.35000000000000003, 64.0], [1.0, 0.45, 64.0], [0.5, 0.15000000000000002, 63.9], [3.5, 0.2, 63.6], [3.0, 0.30000000000000004, 63.5], [0.5, 0.5, 63.4], [2.0, 0.5, 63.4], [2.5, 0.05, 63.3], [2.0, 0.2, 63.3], [4.5, 0.4, 63.2], [4.5, 0.45, 63.2], [2.5, 0.15000000000000002, 63.1], [4.5, 0.05, 63.0], [3.5, 0.4, 63.0], [3.0, 0.1, 62.9], [5.0, 0.5, 62.9], [0.5, 0.4, 62.8]]

Ok, so I have the 20 best set, let's check them with a bit more trials

In [20]:
game = MyHangman()

In [23]:
M = len(top_20)
#MyHangman(weight_count = 0.25, weight_length = 2) best performance so far...

tr = 0
top_20_checked = []
for item in top_20:
    wl = item[0]
    wc = item[1]
    tr += 1
    game.change_weights(wl, wc)
    wincount = 0
    N = 5000
    for i in range(N):
        if i % (N/10) == 0:
            print("on game %i of %i" % (i+1, N))
            win_percent = 100*wincount/(i+1)
            print("win percentage: %f" % win_percent)
        game.start_game(verbose=False)
        wincount += game.get_winstate()
    print("Finished trial %i of %i" % (tr, M))
    print("win percentage: ", 100*wincount/N, "%")
    print()
    top_20_checked.append([wl, wc, 100*wincount/N])

on game 1 of 5000
win percentage: 0.000000
on game 501 of 5000
win percentage: 53.293413
on game 1001 of 5000
win percentage: 52.047952
on game 1501 of 5000
win percentage: 53.164557
on game 2001 of 5000
win percentage: 53.673163
on game 2501 of 5000
win percentage: 53.418633
on game 3001 of 5000
win percentage: 53.148950
on game 3501 of 5000
win percentage: 52.499286
on game 4001 of 5000
win percentage: 51.487128
on game 4501 of 5000
win percentage: 51.433015
Finished trial 1 of 20
win percentage:  51.5 %

on game 1 of 5000
win percentage: 0.000000
on game 501 of 5000
win percentage: 53.692615
on game 1001 of 5000
win percentage: 55.544456
on game 1501 of 5000
win percentage: 53.364424
on game 2001 of 5000
win percentage: 53.773113
on game 2501 of 5000
win percentage: 52.778888
on game 3001 of 5000
win percentage: 53.182273
on game 3501 of 5000
win percentage: 52.442159
on game 4001 of 5000
win percentage: 52.486878
on game 4501 of 5000
win percentage: 52.766052
Finished trial 2 of 20

In [43]:
top_20_checked.sort(key = lambda x: x[2], reverse = True)
for item in top_20_checked:
    print(item)

[4.2, 0.30000000000000004, 53.34]
[3.4000000000000004, 0.30000000000000004, 53.1]
[4.2, 0.1, 53.02]
[5.0, 0.26, 52.9]
[4.6, 0.5, 52.9]
[3.0, 0.18, 52.74]
[4.2, 0.22, 52.48]
[1.4, 0.22, 52.32]
[2.6, 0.45999999999999996, 52.22]
[1.4, 0.45999999999999996, 52.18]
[3.0, 0.30000000000000004, 52.02]
[5.0, 0.18, 51.68]
[1.0, 0.42000000000000004, 51.66]
[1.0, 0.14, 51.66]
[1.8, 0.30000000000000004, 51.62]
[4.2, 0.45999999999999996, 51.5]
[3.0, 0.45999999999999996, 51.5]
[4.2, 0.26, 51.46]
[1.4, 0.42000000000000004, 51.44]
[2.6, 0.26, 51.26]
